In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm
import pickle
from socceraction.data.statsbomb import StatsBombLoader
from statsbombpy import sb

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)
from express.databases import SQLiteDatabase
from express.visualization import plot_action
from express.utils import add_names
from functools import partial

from express.datasets import PressingDataset
from tqdm import tqdm
import pandas as pd
from pathlib import Path

import shap
import matplotlib.pyplot as plt


base_path: c:\Users\mirun\24_winter\press\sr-press


In [2]:
model = "xgboost" #"soccermap"
trial = 13 # as you like

path = Path(base_path) / "stores" / "model" / model / f"{trial:03d}" / "component.pkl"
with path.open(mode="rb") as file:
    component = pickle.load(file)
component

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")
test_db = SQLiteDatabase(TEST_DB_PATH)
dataset_test = partial(PressingDataset, path=os.path.join(base_path, "stores", "filtered_datasets", "test"))


In [11]:
print(component.features.keys())
component.test(dataset_test)

dict_keys(['closest_11_players', 'relative_startlocation', 'defenders_in_3m_radius', 'relative_endlocation', 'get_column_sum_to_player'])


{'precision': 0.4098360655737705,
 'recall': 0.08741258741258741,
 'f1': 0.1440922190201729,
 'log_loss': 0.6164863893636872,
 'brier': 0.21067506735907512,
 'roc_auc': 0.5854128411892265}

In [10]:
preds = component.predict(dataset_test)
preds

game_id  action_id
3857254  4            0.116563
         14           0.006936
         17           0.154682
         22           0.300543
         34           0.133272
                        ...   
3869685  3013         0.072382
         3020         0.301446
         3031         0.269038
         3041         0.212329
         3045         0.173190
Length: 14504, dtype: float32

In [15]:
test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "test"),
    xfns=["closest_11_players", "relative_startlocation", "defenders_in_3m_radius", "relative_endlocation", "get_column_sum_to_player"],
    yfns=["possession_change_by_5_seconds"],
    load_cached=True,
    nb_prev_actions=3
)

component.label

In [13]:
component.label[0]


'possession_change_by_5_seconds'

In [17]:
new = test_dataset.labels.copy()
new[f"{component.label[0]}_prob"] = preds.values
new


possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  
game_id action_id                                       
3857254 4                                     0.116563  
        14                                    0.006936  
        17                                    0.154682  
        22                                    0.300543  
        34                                    0.133272  
...                                                ...  
3869685 3013                                  0.072382  
        3020                                  0.301446  
        3031                                  0.269038  
        3041                                  0.212329  
        3045                                  0.173190  

[14504 rows x 2 columns]

### pressing action마다 Player_id 찾아서 append하기
- 이것보다 빠르게 구할 수 있는 방법 아신다면 수정해주십시오...
- 50분정도 걸립니다

### 압박 및 탈압박 선수 id 구하기

In [27]:
from tqdm import tqdm
import pandas as pd

# 결과 저장용 리스트
states = []
pressure_states = []
pressured_player_ids = []
pressing_action_ids = []
game_ids = []  # game_id 저장용 리스트 추가

# 테스트용 데이터 로드
for game_id in tqdm(test_db.games().index.unique().tolist(), desc="Processing games"):
    actions = add_names(test_db.actions(game_id)).reset_index()

    # 필터 초기화
    idx = pd.Series([True] * len(actions), index=actions.index)

    if idx.sum() < 1:
        print(f"No actions available for game_id {game_id}")
        continue

    # 홈팀 정보 가져오기
    home_team_id, _ = test_db.get_home_away_team_id(game_id)

    # Group by period and process
    for (game_id, period_id), group in tqdm(
        actions.groupby(["game_id", "period_id"], sort=False),
        total=len(actions["period_id"].unique()),
        desc=f"Processing periods for game_id {game_id}"
    ):
        # 액션 반복
        for action in group.itertuples(index=False):
            if action.type_name == "pressing": 
                continue

            start_time = action.time_seconds
            end_time = start_time + action.duration
            team_id = action.team_id

            # Overlap 조건 설정
            overlap_cond1 = (actions['time_seconds'] <= start_time) & ((actions['time_seconds'] + actions['duration']) >= start_time)
            overlap_cond2 = (actions['time_seconds'] >= start_time) & ((actions['time_seconds'] + actions['duration']) <= end_time)
            overlap = (
                (actions["game_id"] == game_id) &
                (actions["period_id"] == period_id) &
                (actions["team_id"] != team_id) &  # 팀 비교
                overlap_cond1
            )

            # 압박 액션 필터링
            pressing_actions = actions[overlap & idx]

            # 데이터 저장
            for pressing_action in pressing_actions.itertuples(index=False):       
                states.append(action)
                pressure_states.append(pressing_action)
                pressured_player_ids.append(action.player_id)
                pressing_action_ids.append(pressing_action.action_id)
                game_ids.append(game_id)  # game_id 추가

# 압박 데이터 확인
print(f"Number of pressing actions: {len(pressing_action_ids)}")
print(f"Number of pressured players: {len(pressured_player_ids)}")

# 데이터를 DataFrame으로 변환
result_df = pd.DataFrame({
    "game_id": game_ids,  # game_id 컬럼 추가
    "pressing_action_id": pressing_action_ids,
    "pressured_player_id": pressured_player_ids
})

# 결과 확인
print(result_df.head())


Processing games: 100%|██████████| 64/64 [06:10<00:00,  5.79s/it]

Number of pressing actions: 20359
Number of pressured players: 20359
   game_id  pressing_action_id  pressured_player_id
0  3857254                   4                17042
1  3857254                   7                 5651
2  3857254                  14                16554
3  3857254                  16                 3570
4  3857254                  16                 3570


In [ ]:
import express.config as config
# 결과 저장용 리스트
states = []
pressure_states = []
pressured_player_ids = []
pressing_action_ids = []
game_ids = []  # 
game_id=3857254                   
def actionfilter(actions: pd.DataFrame) -> pd.Series:
    is_pressing = (actions.type_id == config.actiontypes.index("pressing"))  # pressing
    is_visible_area_360 = actions["visible_area_360"].notna()  # visible_area_360

    return is_pressing & is_visible_area_360

actions = add_names(test_db.actions(game_id)).reset_index()
# 필터 초기화
idx = actionfilter(actions)

if idx.sum() < 1:
    print(f"No actions available for game_id {game_id}")


# 홈팀 정보 가져오기
home_team_id, _ = test_db.get_home_away_team_id(game_id)

# Group by period and process
for (game_id, period_id), group in tqdm(
    actions.groupby(["game_id", "period_id"], sort=False),
    total=len(actions["period_id"].unique()),
    desc=f"Processing periods for game_id {game_id}"
):
    # 액션 반복
    for action in group.itertuples(index=False):
        if action.type_name == "pressing": 
            continue

        start_time = action.time_seconds
        end_time = start_time + action.duration
        team_id = action.team_id
        # overlap1: if a pressure event appears before a pass, and the pressure’s timestamp plus its duration encompasses the pass’s timestamp, that pass is said to have been made under pressure
        # overlap2: if a pressure event occurs after a pass, but before the end of the pass (as calculated by using its duration), that pass is said to have been received under pressure.
            
        # Overlap 조건 설정
        overlap_cond1 = (actions['time_seconds'] <= start_time) & ((actions['time_seconds'] + actions['duration']) >= start_time)
        overlap_cond2 = (actions['time_seconds'] >= start_time) & ((actions['time_seconds'] + actions['duration']) <= end_time)
        overlap = (
            (actions["game_id"] == game_id) &
            (actions["period_id"] == period_id) &
            (actions["team_id"] != team_id) &
            overlap_cond1
        )

        # 압박 액션 필터링
        pressing_actions = actions[overlap & idx]

        # 데이터 저장
        for pressing_action in pressing_actions.itertuples(index=False):       
            states.append(action)
            pressure_states.append(pressing_action)
            pressured_player_ids.append(action.player_id)
            pressing_action_ids.append(pressing_action.action_id)
            game_ids.append(game_id)  # game_id 추가


Processing periods for game_id 3857254: 100%|██████████| 2/2 [00:11<00:00,  5.68s/it]


TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [ ]:
states = pd.DataFrame.from_records([s._asdict() for s in states]).reset_index(drop=True)
pressure_states = pd.DataFrame.from_records([p._asdict() for p in pressure_states]).reset_index(drop=True)
combined_states = pressure_states + states
combined_states["game_id"] = game_id

### player_name 매핑해서 append하기

In [39]:
result_df.head(10)

,game_id,pressing_action_id,pressured_player_id
0,3857254,4,17042
1,3857254,7,5651
2,3857254,14,16554
3,3857254,16,3570
4,3857254,16,3570
5,3857254,19,44955
6,3857254,23,17042
7,3857254,23,17042
8,3857254,32,5651
9,3857254,35,5655


In [40]:
new.head(10)

possession_change_by_5_actions  posession_change_prob
game_id action_id                                                       
3857254 4                                   False               0.248037
        14                                  False               0.121798
        16                                  False               0.110283
        20                                  False               0.283783
        32                                  False               0.226375
        35                                  False               0.331838
        37                                  False               0.565257
        42                                  False               0.333415
        51                                  False               0.222846
        86                                  False               0.353204

In [18]:
loaded_teams_df=pd.read_csv('WC_test_player_id.csv')
new['player_id']=loaded_teams_df['player_id'].values
new

possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  player_id  
game_id action_id                                                  
3857254 4                                     0.116563      23910  
        14                                    0.006936      35592  
        17                                    0.154682       9236  
        22                                    0.300543      44955  
        34                                    0.133272       6302  
...                                                ...        ...  
3869685 3013                                  0.072382      24778  
        3020                                  0.301446       8519  
        3031                                  0.269038      16308  
        3041                                  0.212329      16308  
        3045                                  0.173190      38718  

[14504 rows x 3 columns]

In [19]:
import json
with open('players_2_id.json', 'r', encoding='utf-8') as file:
    team_mapping = json.load(file)
new['player_name'] = new['player_id'].astype(str).map(team_mapping)
new['xP_difference'] =  new[f"{component.label[0]}"] - new[f"{component.label[0]}_prob"]
new



possession_change_by_5_seconds  \
game_id action_id                                   
3857254 4                                   False   
        14                                  False   
        17                                  False   
        22                                  False   
        34                                  False   
...                                           ...   
3869685 3013                                False   
        3020                                False   
        3031                                False   
        3041                                False   
        3045                                False   

                   possession_change_by_5_seconds_prob  player_id  \
game_id action_id                                                   
3857254 4                                     0.116563      23910   
        14                                    0.006936      35592   
        17                                    0.154682       9236   
        22                                    0.300543      44955   
        34                                    0.133272       6302   
...                                                ...        ...   
3869685 3013                                  0.072382      24778   
        3020                                  0.301446       8519   
        3031                                  0.269038      16308   
        3041                                  0.212329      16308   
        3045                                  0.173190      38718   

                                player_name  xP_difference  
game_id action_id                                           
3857254 4                    Youssef Msakni      -0.116563  
        14                 Anis Ben Slimane      -0.006936  
        17                   Mohamed Dräger      -0.154682  
        22             Aïssa Bilal Laïdouni      -0.300543  
        34                   Kasper Dolberg      -0.133272  
...                                     ...            ...  
3869685 3013              Eduardo Camavinga      -0.072382  
        3020       Dayotchanculle Upamecano      -0.301446  
        3031         Leandro Daniel Paredes      -0.269038  
        3041         Leandro Daniel Paredes      -0.212329  
        3045                 Enzo Fernandez      -0.173190  

[14504 rows x 5 columns]

In [21]:
# 1. 팀당 pressing point 합 구하기
team_sum = new.groupby('player_name')[f"{component.label[0]}_prob"].sum().reset_index()

# 2. 팀당 xP_difference 합 구하기
team_difference = new.groupby('player_name')['xP_difference'].sum().reset_index()

# 3. 팀당 서로 다른 game_id 개수 세기
team_game_count = new.reset_index().groupby('player_name')['game_id'].nunique()
team_game_count_df = team_game_count.reset_index()


# 4. 팀당 서로 다른 action_id 개수 세기
team_action_count = new.reset_index().groupby('player_name')['action_id'].nunique()
team_action_count= team_action_count.reset_index()

# 데이터프레임 병합

merged_df = pd.merge(team_sum, team_difference, on='player_name', how='outer')  # 첫 번째 병합
merged_df = pd.merge(merged_df, team_game_count, on='player_name', how='outer')  # 두 번째 병합
merged_df = pd.merge(merged_df, team_action_count, on='player_name', how='outer')  # 세 번째 병합
merged_df.rename(columns={'game_id': 'games_played'}, inplace=True)
merged_df.rename(columns={'action_id': 'actions'}, inplace=True)
merged_df.rename(columns={f"{component.label[0]}_prob": 'xP_total'}, inplace=True)

# 추가 컬럼 계산
merged_df['xP_total']=merged_df['xP_total'].round(2)
merged_df['xP_difference']=merged_df['xP_difference'].round(2)
merged_df['xP_per_game'] = (merged_df['xP_total'] / merged_df['games_played']).round(2)
merged_df['xP_per_action'] = (merged_df['xP_total'] / merged_df['actions']).round(2)


column_order = [
    'player_name',             # 선수 이름
    'games_played',            # 총 게임 수
    'actions',                 # 총 action_id 수    
    'xP_total',                # pressing point 합
    'xP_difference',           # xP_difference
    
    'xP_per_game',             # 경기당 xP
    'xP_per_action'            # action당 xP
]

# 열 순서를 재배치
merged_df = merged_df[column_order]


#merged_df.to_csv('WC_test_player_xP.csv', index=False)

In [23]:
# 정렬
condition=merged_df['games_played']>=3
condition_merged_df = merged_df[condition]
condition_merged_df = condition_merged_df.sort_values('xP_difference', ascending=True)
condition_merged_df.head(10)

,player_name,games_played,actions,xP_total,xP_difference,xP_per_game,xP_per_action
254,Iñaki Williams Arthuer,3,28,7.180000,-7.18,2.39,0.26
497,Remo Freuler,4,52,13.540000,-6.54,3.38,0.26
295,Joshua Kimmich,3,32,10.470000,-6.47,3.49,0.33
171,Enner Remberto Valencia Lastra,3,26,7.090000,-6.09,2.36,0.27
367,Luis Gerardo Chávez Magallón,3,37,11.770000,-5.77,3.92,0.32
487,Przemysław Frankowski,4,35,9.710000,-5.71,2.43,0.28
221,Hakim Ziyech,7,111,27.430000,-5.43,3.92,0.25
191,Firas Tariq Nasser Al Albirakan,3,25,7.050000,-5.05,2.35,0.28
403,Mehdi Taremi,3,59,12.990000,-4.99,4.33,0.22
470,Ousmane Dembélé,7,70,16.950001,-4.95,2.42,0.24
